In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00


In [ ]:
import optuna
import pandas as pd
import numpy as np

In [ ]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI','DiabetesPedigreeFunction', 'Age', 'Outcome']


df=pd.read_csv(url,names=columns)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals]=df[cols_with_missing_vals].replace(0,np.nan)

df.fillna(df.mean(),inplace=True)
df.isna().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [ ]:
X=df.drop(columns=['Outcome'],axis=1)
y=df['Outcome']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

print(X_train.shape,y_train.shape)

(614, 8) (614,)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def objective(trials):
  n_estimator=trials.suggest_int('n_estimator',50,250)
  max_depth=trials.suggest_int('max_depth',1,5)

  #model building
  model=RandomForestClassifier(n_estimators=n_estimator, max_depth=max_depth, random_state=42)

  score=cross_val_score(model,X,y,cv=3,scoring='accuracy').mean()

  return score  # Return the accuracy score for Optuna to maximize

**Bayesian-sampler**

In [ ]:
study=optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler()) # We aim to maximize accuracy
study.optimize(objective,n_trials=50) # Run 50 trials to find the best hyperparameters

[I 2024-09-27 12:13:49,826] A new study created in memory with name: no-name-06374141-a59b-4bfd-80c0-fd559433fb9b
[I 2024-09-27 12:13:50,278] Trial 0 finished with value: 0.7604166666666666 and parameters: {'n_estimator': 100, 'max_depth': 3}. Best is trial 0 with value: 0.7604166666666666.
[I 2024-09-27 12:13:51,608] Trial 1 finished with value: 0.7578125 and parameters: {'n_estimator': 196, 'max_depth': 4}. Best is trial 0 with value: 0.7604166666666666.
[I 2024-09-27 12:13:53,350] Trial 2 finished with value: 0.7643229166666666 and parameters: {'n_estimator': 234, 'max_depth': 5}. Best is trial 2 with value: 0.7643229166666666.
[I 2024-09-27 12:13:54,029] Trial 3 finished with value: 0.7578125 and parameters: {'n_estimator': 147, 'max_depth': 4}. Best is trial 2 with value: 0.7643229166666666.
[I 2024-09-27 12:13:54,724] Trial 4 finished with value: 0.73828125 and parameters: {'n_estimator': 167, 'max_depth': 2}. Best is trial 2 with value: 0.7643229166666666.
[I 2024-09-27 12:13:55

In [ ]:
print('on Training Dataset :')
print(f'best accuracy {study.best_value}')
print(f'best params {study.best_params}')

on Training Dataset :
best accuracy 0.76953125
best params {'n_estimator': 202, 'max_depth': 5}


In [ ]:
from sklearn.metrics import accuracy_score

best_model=RandomForestClassifier(n_estimators=202,max_depth=5,random_state=42)
best_model.fit(X_train,y_train)

y_pred=best_model.predict(X_test)
print(f'Accuracy on Test Data {accuracy_score(y_test,y_pred):.2f}')

Accuracy on Test Data 0.75


**Random-Sampler**

In [ ]:
study=optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler()) # We aim to maximize accuracy
study.optimize(objective,n_trials=50) # Run 50 trials to find the best hyperparameters

[I 2024-09-27 12:02:22,820] A new study created in memory with name: no-name-a4c6496f-6cae-456f-8e23-2124e3cd83f3
[I 2024-09-27 12:02:23,336] Trial 0 finished with value: 0.7473958333333334 and parameters: {'n_estimator': 124, 'max_depth': 2}. Best is trial 0 with value: 0.7473958333333334.
[I 2024-09-27 12:02:24,132] Trial 1 finished with value: 0.7643229166666666 and parameters: {'n_estimator': 177, 'max_depth': 3}. Best is trial 1 with value: 0.7643229166666666.
[I 2024-09-27 12:02:24,871] Trial 2 finished with value: 0.7369791666666666 and parameters: {'n_estimator': 175, 'max_depth': 2}. Best is trial 1 with value: 0.7643229166666666.
[I 2024-09-27 12:02:25,412] Trial 3 finished with value: 0.7578125 and parameters: {'n_estimator': 112, 'max_depth': 4}. Best is trial 1 with value: 0.7643229166666666.
[I 2024-09-27 12:02:26,268] Trial 4 finished with value: 0.69921875 and parameters: {'n_estimator': 216, 'max_depth': 1}. Best is trial 1 with value: 0.7643229166666666.
[I 2024-09-27

In [ ]:
print('on Training Dataset :')
print(f'best accuracy {study.best_value}')
print(f'best params {study.best_params}')

on Training Dataset :
best accuracy 0.7643229166666666
best params {'n_estimator': 177, 'max_depth': 3}


In [ ]:
best_model=RandomForestClassifier(n_estimators=177,max_depth=3,random_state=42)
best_model.fit(X_train,y_train)

y_pred=best_model.predict(X_test)
print(f'Accuracy on Test Data {accuracy_score(y_test,y_pred):.2f}')

Accuracy on Test Data 0.79


**Grid-Search-Sampler**

In [ ]:
search_space={
    'n_estimator':[50,100,150,200,250],
    'max_depth':[1,2,3,4,5]
}

In [ ]:
study=optuna.create_study(direction='maximize',sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2024-09-27 12:07:49,669] A new study created in memory with name: no-name-50225d77-d602-4a5d-8721-d348e2dbcc9c
[I 2024-09-27 12:07:49,899] Trial 0 finished with value: 0.7395833333333334 and parameters: {'n_estimator': 50, 'max_depth': 2}. Best is trial 0 with value: 0.7395833333333334.
[I 2024-09-27 12:07:50,850] Trial 1 finished with value: 0.69921875 and parameters: {'n_estimator': 150, 'max_depth': 1}. Best is trial 0 with value: 0.7395833333333334.
[I 2024-09-27 12:07:52,643] Trial 2 finished with value: 0.7604166666666666 and parameters: {'n_estimator': 250, 'max_depth': 4}. Best is trial 2 with value: 0.7604166666666666.
[I 2024-09-27 12:07:53,184] Trial 3 finished with value: 0.7591145833333334 and parameters: {'n_estimator': 100, 'max_depth': 4}. Best is trial 2 with value: 0.7604166666666666.
[I 2024-09-27 12:07:53,599] Trial 4 finished with value: 0.7604166666666666 and parameters: {'n_estimator': 100, 'max_depth': 3}. Best is trial 2 with value: 0.7604166666666666.
[I 20

In [ ]:
print('on Training Dataset :')
print(f'best accuracy {study.best_value}')
print(f'best params {study.best_params}')

on Training Dataset :
best accuracy 0.765625
best params {'n_estimator': 200, 'max_depth': 5}


In [ ]:
best_model=RandomForestClassifier(n_estimators=200,max_depth=5,random_state=42)
best_model.fit(X_train,y_train)

y_pred=best_model.predict(X_test)
print(f'Accuracy on Test Data {accuracy_score(y_test,y_pred):.2f}')

Accuracy on Test Data 0.75


OPtuna-Visualisations

In [ ]:
from optuna.visualization import plot_contour,plot_optimization_history,plot_parallel_coordinate,plot_param_importances,plot_slice

In [ ]:
plot_optimization_history(study).show()

In [ ]:
plot_parallel_coordinate(study).show()

In [ ]:
plot_param_importances(study).show()

In [ ]:
plot_slice(study).show()

In [ ]:
plot_contour(study).show()

Dynamic Search Space

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [ ]:
def objective(trial):
  model=trial.suggest_categorical('classifier',['GBF','RFC','svm'])

  if model=='GBF':
    #Gradient Boosting Classifier
    n_estimator=trial.suggest_int('n_estimators', 50, 300)
    learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    max_depth=trial.suggest_int('max_depth', 3, 20)
    min_samples_split=trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10)

    model=GradientBoostingClassifier(
        n_estimators=n_estimator,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        min_samples_split=min_samples_split,
        learning_rate=learning_rate,
        random_state=42
    )

  elif model=='svm':
    #Support Vector Machine
    c=trial.suggest_float('C', 0.1, 100, log=True)
    gamma=trial.suggest_categorical('gamma',['auto','scale'])
    kernel=trial.suggest_categorical('kernel',['linear', 'rbf', 'poly', 'sigmoid'])

    model=SVC(C=c,gamma=gamma,kernel=kernel,random_state=42)

  elif model=='RFC':
    #Random Forest Classifier
    n_estimator = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    model=RandomForestClassifier(
        n_estimators=n_estimator,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        min_samples_split=min_samples_split,
        bootstrap=bootstrap,
        random_state=42
    )

  score=cross_val_score(model,X_train,y_train,cv=3,scoring='accuracy').mean()

  return score

In [ ]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100)

[I 2024-09-27 13:20:31,010] A new study created in memory with name: no-name-2e0d0106-dbfd-486d-8a80-9ac3a9647cd2
[I 2024-09-27 13:20:33,389] Trial 0 finished with value: 0.7573170731707317 and parameters: {'classifier': 'GBF', 'n_estimators': 195, 'learning_rate': 0.02808245187143458, 'max_depth': 17, 'min_samples_split': 10, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.7573170731707317.
[I 2024-09-27 13:20:35,648] Trial 1 finished with value: 0.7670891120675912 and parameters: {'classifier': 'GBF', 'n_estimators': 145, 'learning_rate': 0.012429130883921016, 'max_depth': 11, 'min_samples_split': 9, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.7670891120675912.
[I 2024-09-27 13:20:38,250] Trial 2 finished with value: 0.7443487964291408 and parameters: {'classifier': 'GBF', 'n_estimators': 111, 'learning_rate': 0.1601682277058009, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.7670891120675912.
[I 2024-09-27 13:21:02,931]

In [ ]:
print(f'selected param {study.best_trial.params}')
print(f'selected param {study.best_trial.values}')